In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from datetime import datetime


In [ ]:

# Configurações do banco de dados
DB_HOST = 'localhost'
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASS = '12345678'
paginas = 0
# Função para conectar ao banco de dados
def conectar_db():
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )
    return conn

# Função para buscar notícias
def buscar_noticias(termo, paginas):
    noticias = []

    for i in range(paginas):
        url = f"https://www.google.com/search?q={termo}&tbm=nws&start={i * 10}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
    # print(f"{response.text}")
    
    
        for item in soup.find_all("div", {"class": "X7NTVe"}):
            # print(f"{item.find('h3').text}")
            # print(f"{item.find('a')['href']}")
            titulo = item.find('h3').text
            link = item.find('a')['href']
            noticias.append((titulo, link, datetime.now().date()))
    
    return noticias

# Função para salvar notícias no banco de dados
def salvar_noticias(noticias):
    conn = conectar_db()
    cursor = conn.cursor()
    
    for titulo, url, data in noticias:
        cursor.execute(
            "INSERT INTO noticias (titulo, url, data) VALUES (%s, %s, %s)",
            (titulo, url, data)
        )
    
    conn.commit()
    cursor.close()
    conn.close()

# Main
if __name__ == "__main__":
    termo_de_busca = "playstation"  # Altere o termo de busca conforme necessário      
    while True:        
        try:
            paginas=+1
            noticias = buscar_noticias(termo_de_busca, paginas)
            salvar_noticias(noticias)
            print(f"{paginas}")
            print(f"{len(noticias)} notícias salvas com sucesso.")
        except Exception as e:
            print("Occoreu um erro")

